# Notebook 2: Feature Engineering
## Objectif
- Charger dataset_clean_initial.csv
- Extraire features depuis in_time/out_time: parsing datetime, durées, heures/jours
- Statistiques agrégées par employé (mean/median/std/count)
- Supprimer redondances (corr > 0.95)
- Export dataset_enrichi.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
np.random.seed(42)

# Chemins
csv_path = Path('./csv')
in_out_path = Path('./in_out_time')
outputs_path = Path('./outputs')
outputs_path.mkdir(exist_ok=True)

## 1. Charger dataset_clean_initial.csv

In [2]:
df = pd.read_csv(outputs_path / 'dataset_clean_initial.csv')
print(f'Dataset chargé: {df.shape}')
print(df.head())
print(f'\nColonnes: {df.columns.tolist()}')

Dataset chargé: (4410, 23)
   Attrition     BusinessTravel              Department  DistanceFromHome  \
0          0      Travel_Rarely                   Sales                 6   
1          1  Travel_Frequently  Research & Development                10   
2          0  Travel_Frequently  Research & Development                17   
3          0         Non-Travel  Research & Development                 2   
4          0      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeID  JobLevel                    JobRole  \
0          2  Life Sciences           1         1  Healthcare Representative   
1          1  Life Sciences           2         1         Research Scientist   
2          4          Other           3         4            Sales Executive   
3          5  Life Sciences           4         3            Human Resources   
4          1        Medical           5         1            Sales Executive   

   MonthlyIncome  ...  TotalW

## 2. Charger et parser in_time/out_time

In [3]:
# Charger les fichiers in/out
in_time_df = pd.read_csv(in_out_path / 'in_time.csv')
out_time_df = pd.read_csv(in_out_path / 'out_time.csv')

print(f'in_time: {in_time_df.shape}')
print(f'out_time: {out_time_df.shape}')
print(f'\nPremières colonnes in_time: {in_time_df.columns.tolist()[:5]}')

in_time: (4410, 262)
out_time: (4410, 262)

Premières colonnes in_time: ['EmployeeID', '2015-01-01', '2015-01-02', '2015-01-05', '2015-01-06']


In [4]:
# Parser dates et heures
def extract_time_features(in_df, out_df):
    """Extraire features de temps de travail par employé (heures de travail, heures d'arrivée/départ, taux d'heures supplémentaires, etc.)"""
    features = []
    
    # Colonnes employé (1ère colonne = EmployeeID)
    emp_col = in_df.columns[0]
    employee_ids = in_df[emp_col].unique()
    
    for emp_id in employee_ids:
        in_times = in_df[in_df[emp_col] == emp_id].iloc[0, 1:]
        out_times = out_df[out_df[emp_col] == emp_id].iloc[0, 1:]
        
        # Parser datetime
        in_parsed = pd.to_datetime(in_times, errors='coerce')
        out_parsed = pd.to_datetime(out_times, errors='coerce')
        
        # Durées (en heures)
        durations = (out_parsed - in_parsed).dt.total_seconds() / 3600

        # Heures supplémentaires
        is_heures_supp = durations > 8
        
        # Heures d'arrivée
        arrival_hours = in_parsed.dt.hour
        depart_hours = out_parsed.dt.hour
        
        # Extraire stats 
        emp_features = {
            'EmployeeID': emp_id,
            'work_duration_mean': durations.mean(),
            'arrival_hour_mean': arrival_hours.mean(),
            'depart_hour_mean': depart_hours.mean(),
            'overtime_rate': is_heures_supp.mean(),
            'n_work_days': durations.notna().sum(),
            'attendance_rate': durations.notna().sum() / len(durations),
        }
        
        features.append(emp_features)
    
    return pd.DataFrame(features)

time_features = extract_time_features(in_time_df, out_time_df)
print(f'Time features extraites: {time_features.shape}')
print(time_features.head())
print(f'\nStatistiques:')
print(time_features.describe())

Time features extraites: (4410, 7)
   EmployeeID  work_duration_mean  arrival_hour_mean  depart_hour_mean  \
0           1            7.373651           9.491379         16.900862   
1           2            7.718969           9.470339         17.216102   
2           3            7.013240           9.541322         16.524793   
3           4            7.193678           9.476596         16.693617   
4           5            8.006175           9.461224         17.506122   

   overtime_rate  n_work_days  attendance_rate  
0       0.000000          232         0.888889  
1       0.160920          236         0.904215  
2       0.000000          242         0.927203  
3       0.000000          235         0.900383  
4       0.440613          245         0.938697  

Statistiques:
        EmployeeID  work_duration_mean  arrival_hour_mean  depart_hour_mean  \
count  4410.000000         4410.000000        4410.000000       4410.000000   
mean   2205.500000            7.700792           9.50

## 3. Fusionner avec dataset principal

In [5]:
# EmployeeID est déjà présent dans df (conservé depuis NB1)
print(f'EmployeeID présent dans df: {"EmployeeID" in df.columns}')
print(f'Unique EmployeeID dans df: {df["EmployeeID"].nunique()}')

# Fusionner sur EmployeeID existant
df_enriched = df.merge(time_features, on='EmployeeID', how='left')

print(f'Shape après enrichissement: {df_enriched.shape}')
print(f'Nouvelles colonnes: {time_features.columns.tolist()}')
print(df_enriched.head())

EmployeeID présent dans df: True
Unique EmployeeID dans df: 4410
Shape après enrichissement: (4410, 29)
Nouvelles colonnes: ['EmployeeID', 'work_duration_mean', 'arrival_hour_mean', 'depart_hour_mean', 'overtime_rate', 'n_work_days', 'attendance_rate']
   Attrition     BusinessTravel              Department  DistanceFromHome  \
0          0      Travel_Rarely                   Sales                 6   
1          1  Travel_Frequently  Research & Development                10   
2          0  Travel_Frequently  Research & Development                17   
3          0         Non-Travel  Research & Development                 2   
4          0      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeID  JobLevel                    JobRole  \
0          2  Life Sciences           1         1  Healthcare Representative   
1          1  Life Sciences           2         1         Research Scientist   
2          4          Other           3     

## 4. Imputer time_features manquantes

In [6]:
# Vérifier manquants
print('Valeurs manquantes:')
print(df_enriched.isnull().sum())

Valeurs manquantes:
Attrition                   0
BusinessTravel              0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeID                  0
JobLevel                    0
JobRole                     0
MonthlyIncome               0
NumCompaniesWorked         19
PercentSalaryHike           0
StockOptionLevel            0
TotalWorkingYears           9
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
EnvironmentSatisfaction    25
JobSatisfaction            20
WorkLifeBalance            38
JobInvolvement              0
PerformanceRating           0
work_duration_mean          0
arrival_hour_mean           0
depart_hour_mean            0
overtime_rate               0
n_work_days                 0
attendance_rate             0
dtype: int64


## 5. Identifier redondances (corr > 0.95)


In [7]:
# Identifier colonnes hautement corrélées
numeric_df = df_enriched.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr().abs()
print('Paires avec corrélation > 0.95:')
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if corr_matrix.iloc[i, j] > 0.9:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            corr_val = corr_matrix.iloc[i, j]
            print(f'  {col1} <-> {col2}: {corr_val:.4f}')
            high_corr_pairs.append((col1, col2))
print(f'Total: {len(high_corr_pairs)} paires')

Paires avec corrélation > 0.95:
  work_duration_mean <-> depart_hour_mean: 0.9998
  work_duration_mean <-> overtime_rate: 0.9144
  depart_hour_mean <-> overtime_rate: 0.9147
  n_work_days <-> attendance_rate: 1.0000
Total: 4 paires


## 6. Résumé et export

In [8]:
print('\n=== RÉSUMÉ DATASET ENRICHI ===')
print(f'Shape: {df_enriched.shape}')
print(f'Colonnes: {df_enriched.columns.tolist()}')
print(f'\nTypes:')
print(df_enriched.dtypes)
print(f'\nValeurs manquantes: {df_enriched.isnull().sum().sum()}')


=== RÉSUMÉ DATASET ENRICHI ===
Shape: (4410, 29)
Colonnes: ['Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeID', 'JobLevel', 'JobRole', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'JobInvolvement', 'PerformanceRating', 'work_duration_mean', 'arrival_hour_mean', 'depart_hour_mean', 'overtime_rate', 'n_work_days', 'attendance_rate']

Types:
Attrition                    int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeID                   int64
JobLevel                     int64
JobRole                     object
MonthlyIncome                int64
NumCompaniesWorked         float64


In [9]:
# Export
output_file = outputs_path / 'dataset_enrichi.csv'
df_enriched.to_csv(output_file, index=False)
print(f'\n✓ Dataset enrichi exporté: {output_file}')
print(f'  Shape: {df_enriched.shape}')


✓ Dataset enrichi exporté: outputs\dataset_enrichi.csv
  Shape: (4410, 29)
